In [ ]:
import pandas as pd
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt
import panel as pn
from panel.interact import interact
pn.extension("plotly")
import numpy as np
import seaborn as sns
from MCForecastTools import MCSimulation
from dotenv import load_dotenv
import os
import plotly.express as px
from PIL import Image
import requests
from io import BytesIO

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
Kenya_path = Path('kenyausd final.csv')
Russia_path = Path('russiausd final.csv')
Singapore_path = Path('singaporeusd final.csv')
Ukraine_path = Path('ukraineusd final.csv')
Venezuela_path = Path('venusd final.csv')

In [ ]:
Kenya = pd.read_csv(Kenya_path, index_col='Date', parse_dates=True, infer_datetime_format=True)
Russia= pd.read_csv(Russia_path, index_col='Date', parse_dates=True, infer_datetime_format=True)
Singapore = pd.read_csv(Singapore_path, index_col='Date', parse_dates=True, infer_datetime_format=True)
Ukraine = pd.read_csv(Ukraine_path, index_col='Date', parse_dates=True, infer_datetime_format=True)
Venezuela = pd.read_csv(Venezuela_path, index_col='Date', parse_dates=True, infer_datetime_format=True)

In [ ]:
btc=pd.read_csv("BITSTAMP_BTCUSD, 1D.csv", index_col="time", infer_datetime_format=True, parse_dates=True)
eth=pd.read_csv("COINBASE_ETHUSD, 1D.csv", index_col="time", infer_datetime_format=True, parse_dates=True)
ltc=pd.read_csv("COINBASE_LTCUSD, 1D.csv", index_col="time", infer_datetime_format=True, parse_dates=True)
dash=pd.read_csv("KRAKEN_DASHUSD, 1D.csv", index_col="time", infer_datetime_format=True, parse_dates=True)
xmr=pd.read_csv("KRAKEN_XMRUSD, 1D.csv", index_col="time", infer_datetime_format=True, parse_dates=True)
sp500= pd.read_csv("SP_SPX, 1D.csv", index_col="time", infer_datetime_format=True, parse_dates=True)
dollar = pd.read_csv("TVC_DXY, 1D.csv", index_col="time", infer_datetime_format=True, parse_dates=True)

In [ ]:
btc = btc.drop(columns=["Plot", "Plot.1", "Plot.2", "Plot.3", "Plot.4", "Plot.5", "Plot.6", "Plot.7", "Jaw", "Teeth", "Lips", "RSI",'open', 'high', 'low'])
eth = eth.drop(columns=["Plot", "Plot.1", "Plot.2", "Plot.3", "Plot.4", "Plot.5", "Plot.6", "Plot.7", "Jaw", "Teeth", "Lips", "RSI",'open', 'high', 'low'])
ltc = ltc.drop(columns=["Plot", "Plot.1", "Plot.2", "Plot.3", "Plot.4", "Plot.5", "Plot.6", "Plot.7", "Jaw", "Teeth", "Lips", "RSI",'open', 'high', 'low'])
dash = dash.drop(columns=["Plot", "Plot.1", "Plot.2", "Plot.3", "Plot.4", "Plot.5", "Plot.6", "Plot.7", "Jaw", "Teeth", "Lips", "RSI",'open', 'high', 'low'])
xmr = xmr.drop(columns=["Plot", "Plot.1", "Plot.2", "Plot.3", "Plot.4", "Plot.5", "Plot.6", "Plot.7", "Jaw", "Teeth", "Lips", "RSI",'open', 'high', 'low'])
sp500 = sp500.drop(columns=["Plot", "Plot.1", "Plot.2", "Plot.3", "Plot.4", "Plot.5", "Plot.6", "Plot.7", "Jaw", "Teeth", "Lips", "RSI",'open', 'high', 'low'])
dollar = dollar.drop(columns=["Plot", "Plot.1", "Plot.2", "Plot.3", "Plot.4", "Plot.5", "Plot.6", "Plot.7", "Jaw", "Teeth", "Lips", "RSI",'open', 'high', 'low'])

In [ ]:
crypto_all_no_dates = pd.concat([btc, eth, ltc, dash, xmr], axis=1, keys=["BTC", "ETH", "LTC", "DASH", "XMR"])

In [ ]:
crypto_all = crypto_all_no_dates[~(crypto_all_no_dates.index < '2017-04-21')]
crypto_all.index = crypto_all.index.date
crypto_all = crypto_all.reset_index()
crypto_all = crypto_all.groupby(['index']).sum()
crypto_all = crypto_all.replace(0, np.nan)
crypto_all = crypto_all.dropna()
crypto_all.index.names = ['Date']


sp500 = sp500[~(sp500.index < '2017-04-21')]
sp500.index = sp500.index.date
sp500.index.names = ['Date']

dollar = dollar[~(dollar.index < '2017-04-21')]
dollar.index = dollar.index.date
dollar.index.names = ['Date']

crypto_all

In [ ]:
concat = pd.concat([Kenya, Russia, Singapore, Ukraine, Venezuela], axis=1, keys=["KES", "RUB", "SGD", "UAH", "VES"])
Countries_currencies = concat[~(concat.index < '2017-04-21')] 
Countries_currencies = Countries_currencies.dropna()
Countries_currencies

In [ ]:
# Picking smaller stocks
small_monte_foreign = Countries_currencies[["KES", "UAH", "VES"]]

# Configure a Monte Carlo simulation to forecast 5 year of returns using BTC, ETH, and DASH
MC_small_foreign = MCSimulation(
    portfolio_data = small_monte_foreign,
    weights = [.33, .33, .33],
    num_simulation = 1000,
    num_trading_days = 252*5
)

# Print the simulatino input data
MC_small_foreign.portfolio_data.head()

In [ ]:
# Run a Monte Carlo Simulation to show five years of returns
MC_small_foreign.calc_cumulative_return()

In [ ]:
# Simulation outcomes
line_plot_foreign = MC_small_foreign.plot_simulation()

In [ ]:
# Probability distribution
dist_plot_foreign = MC_small_foreign.plot_distribution()

In [ ]:
# Fetch sumary statistics
foreign_summ = MC_small_foreign.summarize_cumulative_return()
print(foreign_summ)

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $10,000 investments in stocks
even_ci_lower = round(foreign_summ[8]*10000,2)
even_ci_upper = round(foreign_summ[9]*10000,2)

# Print results
output_foreign_msg = f"There is a 95% chance that an initial investment of $10,000 in a portfolio of these foreign currencies over the next 5 years will end within in the range of ${even_ci_lower} and ${even_ci_upper}."
output_foreign_msg 

In [ ]:
# Picking smaller stocks
small_monte = crypto_all[["LTC", "DASH", "XMR"]]

# Configure a Monte Carlo simulation to forecast 5 year of returns using BTC, ETH, and DASH
MC_small = MCSimulation(
    portfolio_data = small_monte,
    weights = [.33, .33, .33],
    num_simulation = 1000,
    num_trading_days = 252*5
)

# Print the simulatino input data
MC_small.portfolio_data.head()

In [ ]:
# Run a Monte Carlo Simulation to show five years of returns
MC_small.calc_cumulative_return()

In [ ]:
small_line = MC_small.plot_simulation()

In [ ]:
# Simulation outcomes
small_plot = MC_small.plot_distribution()

In [ ]:
# Fetch sumary statistics
small_summ = MC_small.summarize_cumulative_return()
print(small_summ)

In [ ]:
# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $10,000 investments in stocks
lower_small = round(small_summ[8]*10000,2)
upper_small = round(small_summ[9]*10000,2)

# Print results
output_crypto_msg = f"There is a 95% chance that an initial investment of $10,000 in a portfolio of these cryptocurrencies over the next 5 years will end within in the range of ${lower_small} and ${upper_small}."
output_crypto_msg

In [ ]:
# Daily returns already calculated previously
daily_returns = crypto_all.pct_change()
daily_returns = daily_returns.dropna()

# Calculate annualized volatility of stocks
volatility = daily_returns.std() * np.sqrt(252)
volatility.sort_values(inplace=True)

In [ ]:
volatility_plot = volatility.plot(kind='bar',figsize=(20,10), title = "Cryptocurrency Volatility", xlabel="Cryptocurrencies", ylabel= "Volatility", color="khaki", rot = 0)
print("The most volatile stocks you could purchase would be Litecoin, Dash, and Monero")

In [ ]:
Countries = Countries_currencies.plot(figsize=(20,10), xlabel = "Date", ylabel = "Currency in USD", title = "Foreign Currency Performance")
Countries.legend(['Kenyan Shilling', 'Russian Ruble', 'Singapore Dollar', 'Ukranian Hryvnia', 'Venezuelan Bolívar'])

#All currencies

In [ ]:
Venplot = Venezuela.plot(color = "Black", kind='line', xlabel = "Date", ylabel = "Currency in USD", figsize=(20,10), title = "Hyperflation of the Venezuelan Bolivar")
Venplot.legend(['Venezuelan Bolivar'])

#Venezuela only

In [ ]:
Russia_Ukr = pd.concat([Russia, Ukraine], axis=1)
Russia_Ukr.dropna()
columns = ['Ruble', 'Hryvnia']
Russia_Ukr.columns = columns
Russia_Ukr.plot(figsize=(20,10), xlabel = "Date", ylabel = "Currency in USD", title = "2022 Russian invasion of Ukraine", color =("black","blue"))

#russia/ukraine only

In [ ]:
Kenyaplot = Kenya.plot(figsize=(20,10), xlabel = "Date", ylabel = "Currency in USD", title = "Kenya, a Developing Nation", color = 'black')
Kenyaplot.legend(['Kenyan Shilling'])

#Kenya only

In [ ]:
crypto_plot = crypto_all.plot(figsize=(20,10), xlabel = "Date", ylabel = "Currency in USD", title = "Cryptocurrency Performance")
crypto_plot.legend(['Bitcoin', 'Ethereum', 'Litecoin', 'Dash', 'Monero'])

#General Overview

In [ ]:
crypto_no_bitcoin = pd.concat([eth, ltc, dash, xmr], axis=1, keys=["ETH", "LTC", "DASH", "XMR"])
crypto_no_bitcoin_dates = crypto_no_bitcoin[~(crypto_no_bitcoin.index < '2017-04-21')] 

In [ ]:
crypto_no_bitcoin_dates_plot = crypto_no_bitcoin_dates.plot(figsize=(20,10), xlabel = "Date", ylabel = "Currency in USD", title = "Scaled Down Cryptocurrency Performance")
crypto_no_bitcoin_dates_plot.legend(['Ethereum', 'Litecoin', 'Dash', 'Monero'])


#Can explain that Dash was as popular as Ethereum during their early days; however Ethereum exploded with NFTs

In [ ]:
crypto_just_3_lowest = pd.concat([ltc, dash, xmr], axis=1, keys=["LTC", "DASH", "XMR"])
crypto_just_3_lowest_dates = crypto_just_3_lowest[~(crypto_just_3_lowest.index < '2017-04-21')] 
crypto_just_3_lowest_dates_plot = crypto_just_3_lowest_dates.plot(figsize=(20,10), xlabel = "Date", ylabel = "Currency in USD", title = "Moderate Performing Cryptocurrencies")
crypto_just_3_lowest_dates_plot.legend(['Litecoin', 'Dash', 'Monero'])

#can use side by side with 3 developing to show how much more volatility

In [ ]:
developing = pd.concat([Kenya, Ukraine, Venezuela], axis=1, keys=["KES", "UAH", "VES"])
Countries_currencies_clean = developing[~(developing.index < '2017-04-21')] 
Countries_currencies_plot = Countries_currencies_clean.plot(figsize=(20,10), xlabel = "Date", ylabel = "Currency in USD", title = "Developing Currency Performance")
Countries_currencies_plot.legend(['Kenyan Shilling', 'Ukranian Hryvnia', 'Venezuelan Bolívar'])

#can use side by side with 3 crypto to show how little change is seen with these currencies

In [ ]:
df = pd.read_csv("World Usage.csv").drop_duplicates()
data_to_plot = df[["Country", "Percentage of population", "Latitude", "Longitude"]]
df.index = df['Country']

y=df['Percentage of population']
bar = df['Percentage of population'].nlargest(n=5).plot(color = "khaki", kind='bar', xlabel = "Country Name", ylabel = "Percentage of Population", figsize=(20,10), title = "Top 5 Nations With Cryptocurrency Ownership Based on Percentage Population", rot='horizontal')
bar.text(0,y[0],"12.73%",ha="center",va="bottom")
bar.text(1,y[1],"11.91%",ha="center",va="bottom")
bar.text(2,y[2],"10.34%",ha="center",va="bottom")
bar.text(3,y[3],"9.4%",ha="center",va="bottom")
bar.text(4,y[4],"8.52%",ha="center",va="bottom")


In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")

# Set the Mapbox API
px.set_mapbox_access_token(map_box_api)

In [ ]:
# Plot Data
map_plot = px.scatter_mapbox(
    data_to_plot,
    lat="Latitude",
    lon="Longitude",
    size="Percentage of population",
    color="Country",
    zoom=0.2,
    mapbox_style = 'satellite',
    height = 1000,
    width = 1000,
    title = "Percentage of Crypto Owners by Country"
)

# Display the map
map_final = map_plot.show()
map_final

In [ ]:
dollar.index = pd.to_datetime(dollar.index, utc=True) 
Countries_currencies.index = pd.to_datetime(Countries_currencies.index, utc=True)
country_usd = pd.concat([Countries_currencies,dollar], axis=1)
country_usd.dropna()
country_usd_corr = country_usd.corr()
x_labels = ['KES', 'RUB', 'SDG', 'UAH', 'VES', 'US Dollar Index']
country_heatmap = sns.heatmap(country_usd_corr, vmin=-1, vmax=1, xticklabels = x_labels, yticklabels = x_labels)
country_heatmap.set_title('Correlation Between Foreign Currencies and the U.S. Dollar Index')
country_heatmap.set_xlabel('Currency')
country_heatmap.set_ylabel('Currency')


country_usd_corr
#can explain how russia and ukraine have been correlated (2nd box of KES) or how Kenya is somehow a little correlated, idk

In [ ]:
crypto_all.index = pd.to_datetime(crypto_all.index, utc=True) 
dollar.index = pd.to_datetime(dollar.index, utc=True) 
Countries_currencies.index = pd.to_datetime(Countries_currencies.index, utc=True)
crypto_Countries_index = pd.concat([crypto_all,Countries_currencies, dollar], axis=1)
crypto_Countries_index.dropna()
all_correlation_index = crypto_Countries_index.corr()
x_labels_for_index = ['BTC','ETH','LTC','DASH','XMR','KES', 'RUB', 'SDG', 'UAH', 'VES', 'U.S. Dollar Index']
all_currency_map_index = sns.heatmap(all_correlation_index, vmin=-1, vmax=1, xticklabels = x_labels_for_index, yticklabels = x_labels_for_index)
all_currency_map_index.set_title('Correlation Between Foreign Currencies, Cryptocurrencies, and the U.S. Dollar Index')
all_currency_map_index.set_xlabel('Currency')
all_currency_map_index.set_ylabel('Currency')
#idk what to do with this, maybe that btc and eth are related to sp500 

In [ ]:
crypto_all.index = pd.to_datetime(crypto_all.index, utc=True) 
sp500.index = pd.to_datetime(sp500.index, utc=True) 
crypto_sp500 = pd.concat([crypto_all,sp500], axis=1)
crypto_sp500.dropna()
all_correlation = crypto_sp500.corr()
x_labels_for_all = ['BTC','ETH','LTC','DASH','XMR','S&P 500']
crypto_sp500_map = sns.heatmap(all_correlation, vmin=-1, vmax=1, xticklabels = x_labels_for_all, yticklabels = x_labels_for_all)
crypto_sp500_map.set_title('Correlation Between S&P 500 and Cryptocurrency')
crypto_sp500_map.set_xlabel('Currency')
crypto_sp500_map.set_ylabel('Currency')


#idk what to do with this

In [ ]:
#import images

foreign_monte_img = requests.get('https://i.gyazo.com/8e86dc3887d4e5c6e49c71228357a220.png')
foreign_img = Image.open(BytesIO(foreign_monte_img.content))
foreign_print = output_foreign_msg
foreign_rows = pn.Row(foreign_img, foreign_print)

crypto_monte_img = requests.get('https://i.gyazo.com/93b0ad1f5f52dc9978c8691db482b097.png')
crypto_img = Image.open(BytesIO(crypto_monte_img.content))
crypto_print = output_crypto_msg
crypto_rows = pn.Row(crypto_img, crypto_print)

country_heatmap_img = requests.get('https://i.gyazo.com/2dea056ef734a2c78c7f3ef5a91f1fd9.png')
country_heatmap_imgfinal = Image.open(BytesIO(country_heatmap_img.content))
country_heatmap_text = "We see that the U.S. Dollar Index, determined by other foreign major currencies, is strongly negatively correlated with the Singapore Dollar. When the U.S. Dollar Index appreciates, the Singapore Dollar is expected to depreciate, and vice verca."
country_heatmap_rows = pn.Row(country_heatmap_imgfinal, country_heatmap_text)

crypto_sp500_map_img = requests.get('https://i.gyazo.com/85031d40549fd3bd9a3e3534938ad13b.png')
crypto_sp500_map_imgfinal = Image.open(BytesIO(crypto_sp500_map_img.content))
crypto_sp500_map_text = "We see that the S&P 500, an index of major American companies, is strongly positively correlated with Bitcoin. When the value of Bitcoin appreciates, the S&P 500 is expected to appreciate, and vice verca."
crypto_sp500_map_rows = pn.Row(crypto_sp500_map_imgfinal, crypto_sp500_map_text)


all_currency_map_index_img = requests.get('https://i.gyazo.com/fce34dc3378dce0e5702749880825467.png')
all_currency_map_index_imgfinal = Image.open(BytesIO(all_currency_map_index_img.content))
all_currency_map_index_text = "We see that the Kenyan Shilling is strongly negatively correlated with Bitcoin. When the value of Bitcoin appreciates, the Kenyan Shilling is expected to depreciate, and vice verca."
all_currency_map_index_rows = pn.Row(all_currency_map_index_imgfinal, all_currency_map_index_text)

tabs=pn.Tabs(
("Foreign Monte Carlo", foreign_rows),
("Crypto Monte Carlo", crypto_rows),
("Correlation between foreign currencies and U.S. Dollar Index", country_heatmap_rows),
("Correlation between cryptocurrencies and S&P 500", crypto_sp500_map_imgfinal),
("Correlation between all currencies and U.S. Dollar Index", all_currency_map_index_imgfinal)
)

tabs